In [ ]:
#Importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#fetching data
from sklearn import datasets
cal_housing=datasets.fetch_california_housing()
cal_housing

In [ ]:
#Separate input and output data
df_x=pd.DataFrame(cal_housing.data,columns=cal_housing.feature_names)
y=pd.DataFrame(cal_housing.target,columns=["target"])

In [ ]:
#Displaying input dataset
df_x

In [ ]:
#Check for Null values
print(df_x.isnull().sum())
print(y.isnull().sum())
y

In [ ]:
#check datatypes of each column
df_x.dtypes

In [ ]:
#There are no null values present based on below output
#Concatenate the dataframes
df_cal=pd.concat([df_x,y],axis=1)
df_cal

In [ ]:
#Checking for correlation 
df_cal.corr()

#Here we observe that target variable has a good correaltion with MedInc variable

In [ ]:
#Describe the dataset

df_cal.describe()

#From output it is observed in population column there are some outliers as we saw a significant difference between mean and median


In [ ]:
#Removing outliers
from scipy.stats import zscore
print("with outliers:{}".format(df_cal.shape))
z_scores=zscore(df_cal)
df_cal2=df_cal.loc[(abs(z_scores)<3).all(axis=1)]
print("without outliers:{}".format(df_cal2.shape))


In [ ]:
#Separate the dataset into input and output variable
df_x=df_cal2.drop(columns=["target", "Latitude", "Longitude", "MedInc"])
y=df_cal2[["target"]]

In [ ]:
#Check for skewness
df_x.skew()

In [ ]:
#Treat the skewness
for index in df_x.skew().index:
    if df_x.skew().loc[index]>0.5:
        df_x[index]=np.log(df_x[index])

In [ ]:
#Again check the skewness
df_x.skew()
# The skew drop from 5.7 to 0.4

In [ ]:
df_x

In [ ]:
df_x.isnull().sum(axis = 0)

In [ ]:
#Using Standard Scaler to bring all features to common scale 
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(df_x)
x=sc.transform(df_x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20,random_state=42)


In [ ]:
#Using Linear Regression
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)



In [ ]:
plt.scatter(y_test,y_pred, color = "yellow")
plt.plot(range(0,5),range(5))
plt.xlabel("Y_test")
plt.ylabel("Y_preict")
plt.title("Y_test vs Y_predict")
plt.show()

In [ ]:
#Hold Out Validation 
from sklearn.metrics import r2_score
from timeit import default_timer as timer
start_val = timer()
print("r2_score is: {}".format(r2_score(y_test,y_pred)))
end_val = timer()
print("Time taken for hold out validation :{}".format(end_val-start_val))

In [ ]:
#Using Cross validatiion 
from sklearn.model_selection import cross_val_score
start_cross_val = timer()
c_scores=cross_val_score(LinearRegression(),x,y,cv=5,scoring="r2")
print("mean r2 score: {}".format(c_scores.mean()))
print("std dev in r2 score: {}".format(c_scores.std()))
end_cross_val = timer()
print("scores :{}".format(c_scores))
print("Time taken for cross validation :{}".format(end_cross_val-start_cross_val))

In [ ]:
x.shape[0]

In [ ]:
#LOOCV: Leave one out cross validation 
#Here partitions are equal to number of instances in dataset
start_loocv_val = timer()
c_scores=cross_val_score(LinearRegression(),x,y,cv=5,scoring='neg_mean_squared_error')
print("mean r2 score: {}".format(c_scores.mean()))
print("std dev in r2 score: {}".format(c_scores.std()))
end_loocv_val = timer()
print("Time taken for LOO cross validation :{}".format(end_loocv_val-start_loocv_val))